# intento 1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import plotly.graph_objs as go
from dash import Dash, dcc, html, Input, Output, State

# -------------------- PARTE 1: MODELO DE PREDICCION -------------------- #

# Parámetros
nombre_csv = 4160  # Nombre base del archivo CSV
timesteps = 24  # Número de pasos temporales

# Cargar el modelo guardado
try:
    model_cargado = load_model(f'entrenamientos/{nombre_csv}modelo_entrenado.h5')
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    exit()

# Cargar los datos
df = pd.read_csv(f'Datos_prueba2/{nombre_csv}_p.csv')

# Seleccionar características para predicción
X_test = df[['period eating', 'period other', 'period resting', 'period rumination']]

# Validar si hay suficientes datos para estandarización
try:
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
except Exception as e:
    print(f"Error al escalar los datos: {e}")
    exit()

# Validar que hay suficientes datos para crear secuencias
if len(X_test_scaled) > timesteps:
    # Crear secuencias temporales
    X_test_reshaped = np.array([X_test_scaled[i:i + timesteps] for i in range(len(X_test_scaled) - timesteps)])
    print("Forma de X_test_reshaped:", X_test_reshaped.shape)
    
    # Predecir las etiquetas
    try:
        y_pred_proba = model_cargado.predict(X_test_reshaped)
        y_pred_classes = np.argmax(y_pred_proba, axis=1)
        print("Clases predichas:", y_pred_classes)

        # Identificar horas con niveles de estés alerta y peligro
        niveles_estres = ['Normal', 'Alerta', 'Peligro']
        horas_estres = [(i, niveles_estres[clase]) for i, clase in enumerate(y_pred_classes) if clase > 0]
        print("Horas con niveles de estés alerta y peligro:", horas_estres)
    except Exception as e:
        print(f"Error en la predicción: {e}")
else:
    print("No hay suficientes datos para generar secuencias de 24 timesteps.")

# -------------------- PARTE 2: DASHBOARD INTERACTIVO -------------------- #

# Funciones auxiliares para generar datos simulados
def cargar_datos(fecha_inicio, dias=7):
    """
    Genera datos simulados de actividad real y predicha
    """
    fechas = pd.date_range(start=fecha_inicio, periods=24*dias, freq='H')
    actividad_real = np.sin(np.linspace(0, 2*np.pi*dias, len(fechas))) + np.random.normal(0, 0.1, len(fechas))
    prediccion = np.sin(np.linspace(0, 2*np.pi*dias, len(fechas)))
    df = pd.DataFrame({'Fecha': fechas, 'Actividad Real': actividad_real, 'Prediccion': prediccion})
    return df

def agregar_anotaciones(figura, horas_estres):
    """
    Agrega anotaciones en las horas con niveles de estrés alerta y peligro.
    """
    for hora, nivel in horas_estres:
        figura.add_annotation(
            x=figura['data'][0]['x'][hora], 
            y=figura['data'][0]['y'][hora],
            text=nivel, 
            showarrow=True, 
            arrowhead=2, 
            font=dict(color='red')
        )

# Crear la app de Dash
app = Dash(__name__)
initial_days = 7
fecha_inicial = "2024-07-01"
df = cargar_datos(fecha_inicial, initial_days)

# Datos de ejemplo de estrés
horas_estres = [(50, 'Alerta'), (100, 'Peligro')]

# Layout de la app
app.layout = html.Div([
    html.H1("Gráfica de Actividad y Predicción", style={'textAlign': 'center'}),
    dcc.Tabs([
        dcc.Tab(label='Datos Analizados y Comparados', children=[
            dcc.Graph(id='grafica-comparada')
        ]),
        dcc.Tab(label='Gráfica Predicha', children=[
            dcc.Graph(id='grafica-predicha')
        ]),
    ]),
    html.Button("Cargar Más Datos", id='boton-cargar', n_clicks=0, style={'margin': '10px'}),
])

# Callback para actualizar las gráficas
@app.callback(
    [Output('grafica-comparada', 'figure'), Output('grafica-predicha', 'figure')],
    Input('boton-cargar', 'n_clicks')
)
def actualizar_grafico(n_clicks):
    dias_extra = 7 * (n_clicks + 1)
    df_actualizado = cargar_datos(fecha_inicial, dias_extra) 

    # Filtrar únicamente los últimos 7 días
    df_reciente = df_actualizado.tail(24 * 7)

    # Crear figura para datos analizados y comparados
    fig_comparada = go.Figure()
    fig_comparada.add_trace(go.Scatter(x=df_reciente['Fecha'], y=df_reciente['Actividad Real'],
                                       mode='lines', name='Actividad Real'))
    fig_comparada.add_trace(go.Scatter(x=df_reciente['Fecha'], y=df_reciente['Prediccion'],
                                       mode='lines', name='Predicción', line=dict(dash='dot')))
    agregar_anotaciones(fig_comparada, horas_estres)
    fig_comparada.update_layout(
        title="Datos Analizados y Comparados",
        xaxis=dict(title='Fecha', rangeslider=dict(visible=True), type='date'),
        yaxis=dict(title='Nivel de Actividad'),
        margin=dict(l=40, r=40, t=40, b=40),
    )

    # Crear figura para gráfica predicha
    fig_predicha = go.Figure()
    fig_predicha.add_trace(go.Scatter(x=df_reciente['Fecha'], y=df_reciente['Prediccion'],
                                      mode='lines', name='Predicción', line=dict(color='blue')))
    fig_predicha.update_layout(
        title="Gráfica Predicha",
        xaxis=dict(title='Fecha', rangeslider=dict(visible=True), type='date'),
        yaxis=dict(title='Nivel de Actividad'),
        margin=dict(l=40, r=40, t=40, b=40),
    )

    return fig_comparada, fig_predicha

# Ejecutar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)


# intento 2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import plotly.graph_objs as go
from dash import Dash, dcc, html, Input, Output

# -------------------- PARTE 1: MODELO DE PREDICCION -------------------- #

# Parámetros
nombre_csv = 4160  # Nombre base del archivo CSV
timesteps = 24  # Número de pasos temporales

# Cargar el modelo guardado
model_cargado = load_model(f'entrenamientos/{nombre_csv}modelo_entrenado.h5')

# Cargar los datos
df = pd.read_csv(f'Datos_prueba2/{nombre_csv}_p.csv')

# Seleccionar características y predecir
X_test = df[['period eating', 'period other', 'period resting', 'period rumination']]
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Validar que hay suficientes datos para crear secuencias
if len(X_test_scaled) > timesteps:
    # Crear secuencias temporales
    X_test_reshaped = np.array([X_test_scaled[i:i + timesteps] for i in range(len(X_test_scaled) - timesteps)])
    print("Forma de X_test_reshaped:", X_test_reshaped.shape)
    
    # Predecir las etiquetas
    try:
        y_pred_proba = model_cargado.predict(X_test_reshaped)
        y_pred_classes = np.argmax(y_pred_proba, axis=1)
        print("Clases predichas:", y_pred_classes)

        # Identificar horas con niveles de estés alerta y peligro
        niveles_estres = ['Normal', 'Alerta', 'Peligro']
        horas_estres = [(i, niveles_estres[clase]) for i, clase in enumerate(y_pred_classes) if clase > 0]
        print("Horas con niveles de estés alerta y peligro:", horas_estres)
    except Exception as e:
        print(f"Error en la predicción: {e}")
else:
    print("No hay suficientes datos para generar secuencias de 24 timesteps.")

# -------------------- PARTE 2: DASHBOARD INTERACTIVO -------------------- #

# Generar datos de ejemplo
def cargar_datos_completos(fecha_inicio, dias=7):
    """
    Genera datos de actividad real y predicción, con datos reales y simulados.
    """
    fechas_reales = pd.date_range(start=fecha_inicio, periods=24*dias, freq='H')
    actividad_real = np.sin(np.linspace(0, 2*np.pi*dias, len(fechas_reales))) + np.random.normal(0, 0.1, len(fechas_reales))
    
    # Generar predicciones para fechas futuras
    fechas_predichas = pd.date_range(start=fechas_reales[-1] + pd.Timedelta(hours=1), periods=24*3, freq='H')
    prediccion = np.sin(np.linspace(0, 2*np.pi*3, len(fechas_predichas)))
    
    df_reales = pd.DataFrame({'Fecha': fechas_reales, 'Actividad Real': actividad_real, 'Prediccion': None})
    df_predichas = pd.DataFrame({'Fecha': fechas_predichas, 'Actividad Real': None, 'Prediccion': prediccion})
    
    return pd.concat([df_reales, df_predichas], ignore_index=True), fechas_reales[-1]

# Función para agregar etiquetas de estrés
def agregar_anotaciones(figura, df, horas_estres):
    """
    Agrega anotaciones en las horas con niveles de estrés alerta y peligro.
    """
    for hora, nivel in horas_estres:
        figura.add_annotation(
            x=df['Fecha'].iloc[hora],
            y=df['Actividad Real'].iloc[hora] if not pd.isna(df['Actividad Real'].iloc[hora]) else df['Prediccion'].iloc[hora],
            text=nivel,
            showarrow=True,
            arrowhead=2,
            font=dict(color='red', size=10),
            ax=0,
            ay=-30
        )

# Crear la app de Dash
app = Dash(__name__)
fecha_inicial = "2024-07-01"
df_completo, fecha_separacion = cargar_datos_completos(fecha_inicial, 7)

# Definir horas con niveles de estrés
horas_estres = [(50, 'Alerta'), (100, 'Peligro'), (150, 'Alerta')]

app.layout = html.Div([
    html.H1("Gráfica de Actividad y Predicción Continua", style={'textAlign': 'center'}),
    dcc.Graph(id='grafica-continuada')
])

@app.callback(
    Output('grafica-continuada', 'figure'),
    Input('grafica-continuada', 'relayoutData')
)
def generar_grafico(_):
    # Crear la gráfica
    fig = go.Figure()
    
    # Datos reales
    fig.add_trace(go.Scatter(
        x=df_completo['Fecha'], 
        y=df_completo['Actividad Real'], 
        mode='lines', 
        name='Actividad Real', 
        line=dict(color='blue')
    ))

    # Datos predichos
    fig.add_trace(go.Scatter(
        x=df_completo['Fecha'], 
        y=df_completo['Prediccion'], 
        mode='lines', 
        name='Predicción', 
        line=dict(color='red', dash='dot')
    ))

    # Línea de separación
    fig.add_shape(
        dict(
            type="line",
            x0=fecha_separacion, y0=0, x1=fecha_separacion, y1=1,
            yref='paper', line=dict(color="black", dash="dash")
        )
    )

    # Agregar etiquetas de estrés
    agregar_anotaciones(fig, df_completo, horas_estres)

    # Ajustar el diseño
    fig.update_layout(
        title="Datos Analizados (Izquierda) y Predicción (Derecha)",
        xaxis=dict(title='Fecha', rangeslider=dict(visible=True), type='date'),
        yaxis=dict(title='Nivel de Actividad'),
        margin=dict(l=40, r=40, t=40, b=40),
    )
    
    return fig

# Ejecutar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)


# intento 3

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import plotly.graph_objs as go
from dash import Dash, dcc, html, Input, Output
from datetime import datetime
# -------------------- PARTE 1: MODELO DE PREDICCIÓN -------------------- #

# Parámetros
nombre_csv = 4160  # Nombre base del archivo CSV
timesteps = 24  # Número de pasos temporales

# Cargar el modelo guardado
try:
    model_cargado = load_model(f'entrenamientos/{nombre_csv}modelo_entrenado.h5')
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    exit()

# Cargar los datos
try:
    df = pd.read_csv(f'Datos_prueba2/{nombre_csv}_p.csv')
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")
    exit()

# Seleccionar características para predicción
X_test = df[['period eating', 'period other', 'period resting', 'period rumination']]

# Validar si hay suficientes datos para estandarización
try:
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
except Exception as e:
    print(f"Error al escalar los datos: {e}")
    exit()

# Validar que hay suficientes datos para crear secuencias
if len(X_test_scaled) > timesteps:
    # Crear secuencias temporales
    X_test_reshaped = np.array([X_test_scaled[i:i + timesteps] for i in range(len(X_test_scaled) - timesteps)])
    print("Forma de X_test_reshaped:", X_test_reshaped.shape)
    try:
        # Predecir las etiquetas
        y_pred_proba = model_cargado.predict(X_test_reshaped)
        y_pred_classes = np.argmax(y_pred_proba, axis=1)
        print("Clases predichas:", y_pred_classes)

        # Identificar horas con niveles de estrés alerta y peligro
        niveles_estres = ['Normal', 'Alerta', 'Peligro']
        horas_estres = [(df['datetime'].iloc[i], niveles_estres[clase])
                            for i, clase in enumerate(y_pred_classes) if clase > 0]
        # Convert horas_estres to datetime
        #horas_estres = [(datetime.strptime(hora, "%Y-%m-%d %H:%M:%S"), estres) for hora, estres in horas_estres]
        print("Horas con niveles de estrés alerta y peligro:", horas_estres)
    except Exception as e:
        print(f"Error en la predicción: {e}")
        horas_estres = []
else:
    print("No hay suficientes datos para generar secuencias de 24 timesteps.")
    horas_estres = []

# -------------------- PARTE 2: DASHBOARD INTERACTIVO -------------------- #

# Función para generar datos reales
def cargar_datos_reales(fecha_inicio, dias=7):
    fechas_reales = pd.date_range(start=fecha_inicio, periods=24 * dias, freq='h')
    actividad_real = np.sin(np.linspace(0, 2 * np.pi * dias, len(fechas_reales))) + np.random.normal(0, 0.1, len(fechas_reales))
    return pd.DataFrame({'Fecha': fechas_reales, 'Actividad Real': actividad_real, 'Prediccion': None})

# Función para generar datos predichos
def cargar_datos_predichos(fecha_inicio):
    fechas_predichas = pd.date_range(start=fecha_inicio, periods=24 * 3, freq='h')
    prediccion = np.sin(np.linspace(0, 2 * np.pi * 3, len(fechas_predichas)))
    return pd.DataFrame({'Fecha': fechas_predichas, 'Actividad Real': None, 'Prediccion': prediccion})


# Función para agregar anotaciones de estrés al gráfico
# Función para agregar anotaciones de estrés al gráfico
def agregar_anotaciones(fig, df_completo, horas_estres):
    # Filtrar solo las horas de estrés que están presentes en df_completo
    for hora, nivel_estres in horas_estres:
        # Verificar si la hora está en el DataFrame df_completo
        if hora in df_completo['Fecha'].values:
            # Encontrar el índice de la hora en df_completo
            idx = df_completo[df_completo['Fecha'] == hora].index[0]
            
            # Obtener el valor de la actividad real en esa hora
            actividad_real = df_completo.iloc[idx]['Actividad Real']
            
            # Agregar anotación al gráfico
            fig.add_annotation(
                x=hora,
                y=actividad_real,
                text=nivel_estres,
                showarrow=True,
                arrowhead=2,
                ax=0,
                ay=-40,  # Ajuste vertical de la anotación
                font=dict(size=12, color='red'),
                arrowcolor='black',
            )


# Crear la app de Dash
app = Dash(__name__)

# Generar datos iniciales
fecha_inicial = df['Fecha'].max() - pd.Timedelta(days=7)
df_reales = cargar_datos_reales(fecha_inicial, 7)
df_predichos = cargar_datos_predichos(df_reales['Fecha'].max())

# Concatenar datos reales y predichos
df_completo = pd.concat([df_reales, df_predichos], ignore_index=True)

# Layout de la app
app.layout = html.Div([
    html.H1("Gráfica de Actividad y Predicción Continua", style={'textAlign': 'center'}),
    dcc.Graph(id='grafica-continuada'),
    html.Button("Cargar Más Datos", id='boton-cargar', n_clicks=0, style={'margin': '10px'}),
])

# Callback para actualizar el gráfico
@app.callback(
    Output('grafica-continuada', 'figure'),
    Input('boton-cargar', 'n_clicks')
)
def actualizar_grafico(n_clicks):
    global df_completo, fecha_separacion

    # Generar nuevos datos
    fecha_inicio_nueva = df_completo['Fecha'].min() - pd.Timedelta(days=7)
    nuevos_datos = cargar_datos_reales(fecha_inicio_nueva, 7)
    fecha_separacion=df_predichos['Fecha'].min()
    # Concatenar y limpiar
    df_completo = pd.concat([nuevos_datos, df_completo], ignore_index=True).drop_duplicates(subset='Fecha').sort_values('Fecha')

    # Crear la figura
    fig = go.Figure()

    # Gráfica de datos reales
    fig.add_trace(go.Scatter(
        x=df_completo['Fecha'],
        y=df_completo['Actividad Real'],
        mode='lines',
        name='Actividad Real',
        line=dict(color='blue')
    ))

    # Gráfica de datos predichos
    fig.add_trace(go.Scatter(
        x=df_completo['Fecha'],
        y=df_completo['Prediccion'],
        mode='lines',
        name='Predicción',
        line=dict(color='red', dash='dot')
    ))

    # Línea de separación
    fig.add_shape(
        dict(
            type="line",
            x0=fecha_separacion, y0=0, x1=fecha_separacion, y1=1,
            yref='paper', line=dict(color="black", dash="dash")
        )
    )
    
    # Agregar anotaciones de estrés
    agregar_anotaciones(fig, df_completo, horas_estres)

    # Configuración del layout
    fig.update_layout(
        title="Nivel de Actividad y Predicción Continua",
        xaxis=dict(title="Fecha", rangeslider=dict(visible=True), type='date'),
        yaxis=dict(title="Nivel de Actividad"),
        margin=dict(l=40, r=40, t=40, b=40)
    )
    return fig

# Ejecutar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)